In [ ]:
!pip install tensorflow transformers efficientnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.1 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model


In [ ]:
# Créez un tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Créez un modèle BERT non entraînable
bert_model = TFBertModel.from_pretrained('bert-base-uncased', trainable=False)

# Créez un modèle EfficientNet pour les images
image_input = Input(shape=(224, 224, 3))
effnet_model = EfficientNetB5(input_tensor=image_input, include_top=False)
for layer in effnet_model.layers:
    layer.trainable = False  # Définissez toutes les couches comme non entraînables

effnet_features = GlobalAveragePooling2D()(effnet_model.output)

text = "Votre texte ici"

# Utilisez le tokenizer BERT pour convertir le texte en séquences d'indices
text_tokens = tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True, padding='max_length', return_tensors='tf')

# Assurez-vous que les indices sont du type int32
text_tokens = text_tokens.numpy().astype('int32')

# Utilisez ces indices dans le couche d'entrée pour le texte
text_input = Input(shape=(512,), dtype='int32')  # Notez le dtype='int32'
text_embeddings = bert_model(text_input)[1]

# Fusionnez les embeddings textuels et les caractéristiques de l'EfficientNet
merged = Concatenate()([text_embeddings, effnet_features])

# Ajoutez une couche de sortie
output = Dense(1, activation='sigmoid')(merged)

# Créez le modèle complet
multi_input_model = Model(inputs=[text_input, image_input], outputs=output)

# Compilez et résumez le modèle
multi_input_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
multi_input_model.summary()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

115263384/115263384 [==============================] - 1s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_4 (Rescaling)     (None, 224, 224, 3)          0         ['input_6[0][0]']             
                                                                                                  
 normalization_2 (Normaliza  (None, 224, 224, 3)          7         ['rescaling_4[0][0]']         
 tion)                                                                                            
                                                                                                  
 rescaling_5 (Rescaling) 

In [ ]:
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
# Plot the model
plot_model(multi_input_model, show_shapes=True, show_layer_names=True)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=79e57a897cf1793a53aac663b7dce6f1ab4716b6daabb6fc9a9e9f059cab5bc9
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Chargez le fichier CSV dans un DataFrame pandas
df = pd.read_csv("/content/preparation.csv")

# Utilisez train_test_split pour diviser les données de manière stratifiée
X = df.drop(columns=["product_category_tree_target"])
y = df["product_category_tree_target"]




In [ ]:
# X["image"] = X["image"].apply(lambda x: "https://storagevangansberg.blob.core.windows.net/imgs/" + x)

In [ ]:
from sklearn import preprocessing
from keras.utils import to_categorical

label = preprocessing.LabelEncoder()
df["label"]  = label.fit_transform(y)
df["label"]  = to_categorical(df["label"] )


In [ ]:
df

,image,Description,product_category_tree_target,label
0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,Key Features of Elegance Polyester Multicolor ...,Home Furnishing,0.0
1,7b72c92c2f6c40268628ec5f14c6d590.jpg,Specifications of Sathiyas Cotton Bath Towel (...,Baby Care,1.0
2,64d5d4a258243731dc7bbb1eef49ad74.jpg,Key Features of Eurospa Cotton Terry Face Towe...,Baby Care,1.0
3,d4684dcdc759dd9cdf41504698d737d8.jpg,Key Features of SANTOSH ROYAL FASHION Cotton P...,Home Furnishing,0.0
4,6325b6870c54cd47be6ebfbffa620ec7.jpg,Key Features of Jaipur Print Cotton Floral Kin...,Home Furnishing,0.0
...,...,...,...,...
1045,958f54f4c46b53c8a0a9b8167d9140bc.jpg,Oren Empower Extra Large Self Adhesive Sticker...,Baby Care,1.0
1046,fd6cbcc22efb6b761bd564c28928483c.jpg,Wallmantra Large Vinyl Sticker Sticker (Pack o...,Baby Care,1.0
1047,5912e037d12774bb73a2048f35a00009.jpg,Buy Uberlyfe Extra Large Pigmented Polyvinyl F...,Baby Care,1.0
1048,c3edc504d1b4f0ba6224fa53a43a7ad6.jpg,Buy Wallmantra Medium Vinyl Sticker Sticker fo...,Baby Care,1.0


In [ ]:
# Spécifiez les proportions de division souhaitées
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

# Utilisez train_test_split pour diviser les données
X_train, X_temp= train_test_split(df, test_size=test_ratio, stratify=df["label"], random_state=123)

In [ ]:
from tensorflow.keras.utils import load_img,img_to_array
from io import BytesIO

ACCOUNT_NAME = "storagevangansberg"
CONTAINER_NAME = "imgs"

# 1. Define the Image Processing Functions

def load_and_preprocess_image(image_path, target_size=(224, 224)):
    import requests
    # Construct the URL for the image
    blob_url = f"https://{ACCOUNT_NAME}.blob.core.windows.net/{CONTAINER_NAME}/{image_path}"

    # Load the image from Azure Blob Storage into memory
    response = requests.get(blob_url)
    img = load_img(BytesIO(response.content), target_size=target_size)

    img_array = img_to_array(img)
    return img_array

In [76]:
df["vec_img"] = df["image"].apply(lambda x:load_and_preprocess_image(x))

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (93680328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [77]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

text_data_generator = tf.data.Dataset.from_tensor_slices(X_train["Description"])
text_data_generator = text_data_generator.batch(32)  # Batch size can be adjusted

# Define a data generator for images (replace 'image_directory' with the actual path)
image_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
image_generator = image_data_generator.flow_from_dataframe(
    dataframe=X_train,
    x_col="vec_img",
    y_col="product_category_tree_target",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=True
)

# Model Training
history = multi_input_model.fit(
    x=[text_data_generator, image_generator],
    y=df["product_category_tree_target"],
    epochs=5
   )


# Model Evaluation
test_loss, test_accuracy = multi_input_model.evaluate(
    x=[test_text_input_data, test_image_generator],
    y=y_test
)

print("Loss on the test set:", test_loss)
print("Accuracy on the test set:", test_accuracy)

KeyError: ignored